# 📚 Simulation 5: Veri Sözlüğü ile Pipeline

Veri sözlüğü entegrasyonu ve zengin raporlama.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

try:
    from risk_pipeline.pipeline16 import RiskModelPipeline, Config
except:
    import sys
    sys.path.append('..')
    from src.risk_pipeline.pipeline16 import RiskModelPipeline, Config

print("✅ Ready")

In [ ]:
# Türkçe değişken isimleri ile veri oluştur
np.random.seed(2024)

n = 3000
df = pd.DataFrame({
    'musteri_id': [f'M_{i:06d}' for i in range(n)],
    'basvuru_tarihi': pd.date_range('2023-01-01', periods=n, freq='D'),
    'temerrut': np.random.binomial(1, 0.18, n),
    
    # Numerik
    'yas': np.random.randint(18, 70, n),
    'aylik_gelir': np.random.lognormal(10, 0.5, n),
    'kredi_notu': np.random.normal(650, 100, n).clip(300, 850),
    'toplam_borc': np.random.exponential(30000, n),
    'kredi_tutari': np.random.exponential(50000, n),
    'is_tecrubesi': np.random.exponential(5, n).clip(0, 30),
    'bakmakla_yukumlu': np.random.poisson(1.5, n),
    
    # Kategorik
    'egitim_durumu': np.random.choice(['Ilkokul', 'Lise', 'Universite', 'Y.Lisans'], n),
    'calisma_durumu': np.random.choice(['Ozel', 'Kamu', 'Serbest', 'Emekli'], n),
    'medeni_durum': np.random.choice(['Bekar', 'Evli', 'Dul'], n),
    'ev_durumu': np.random.choice(['Kendi_Evi', 'Kira', 'Aile_Yanı'], n),
    'il_grubu': np.random.choice(['Buyuksehir', 'Sehir', 'Ilce'], n)
})

print(f"Veri seti: {df.shape}")
print(f"Temerrüt oranı: {df['temerrut'].mean():.2%}")

In [ ]:
# Detaylı veri sözlüğü oluştur
veri_sozlugu = pd.DataFrame({
    'alan_adi': [
        'yas', 'aylik_gelir', 'kredi_notu', 'toplam_borc', 'kredi_tutari',
        'is_tecrubesi', 'bakmakla_yukumlu', 'egitim_durumu', 'calisma_durumu',
        'medeni_durum', 'ev_durumu', 'il_grubu'
    ],
    'alan_aciklamasi': [
        'Müşterinin yaşı (18-70 arası)',
        'Aylık düzenli gelir tutarı (TL)',
        'Kredi derecelendirme notu (300-850 arası)',
        'Mevcut toplam borç tutarı (TL)',
        'Talep edilen kredi tutarı (TL)',
        'Toplam iş tecrübesi (yıl)',
        'Bakmakla yükümlü olunan kişi sayısı',
        'En son mezun olunan eğitim seviyesi',
        'Mevcut çalışma durumu ve sektörü',
        'Güncel medeni durum',
        'Konut sahiplik/kiracılık durumu',
        'İkamet edilen il grubu (nüfus bazlı)'
    ]
})

print("📚 Veri Sözlüğü Hazır:")
print(veri_sozlugu.to_string(index=False))

In [ ]:
# Veri sözlüğü ile pipeline
cfg = Config(
    id_col='musteri_id',
    time_col='basvuru_tarihi',
    target_col='temerrut',
    
    # Veri sözlüğü
    data_dictionary_df=veri_sozlugu,  # DataFrame olarak
    
    # Model ayarları
    use_test_split=True,
    oot_window_months=2,
    cv_folds=3,
    hpo_timeout=60,
    hpo_n_trials=10,
    
    # Output
    output_folder='outputs_with_dict',
    output_excel_path='model_with_dictionary.xlsx'
)

print("Config with data dictionary ready")

In [ ]:
# Pipeline çalıştır
pipeline = RiskModelPipeline(cfg)
pipeline.run(df)

print("\n" + "="*60)
print("✅ Pipeline Complete")
print("="*60)

In [ ]:
# Veri sözlüğü entegrasyonunu kontrol et
print("📚 Veri Sözlüğü Kontrolü:")
print("="*60)

if pipeline.best_model_vars_df_ is not None:
    print("\nEn İyi Model Değişkenleri (açıklamalar ile):")
    display_df = pipeline.best_model_vars_df_[['variable', 'description', 'coef_or_importance']].head(10)
    print(display_df.to_string())
    
    # Açıklama sayısı
    has_desc = pipeline.best_model_vars_df_['description'].notna().sum()
    total_vars = len(pipeline.best_model_vars_df_)
    print(f"\n✅ {has_desc}/{total_vars} değişken açıklaması yüklendi")

In [ ]:
# WOE raporunu kontrol et
print("\n📊 WOE Raporu (Monotonic Sıralı):")
print("="*60)

if pipeline.best_model_woe_df_ is not None:
    # İlk değişken örneği
    first_var = pipeline.best_model_woe_df_['variable'].iloc[0]
    var_woe = pipeline.best_model_woe_df_[pipeline.best_model_woe_df_['variable'] == first_var]
    
    print(f"Değişken: {first_var}")
    if 'variable_description' in var_woe.columns:
        desc = var_woe['variable_description'].iloc[0]
        if desc:
            print(f"Açıklama: {desc}")
    
    print("\nWOE Binleri (event_rate sıralı):")
    display_cols = ['group', 'bin_from', 'bin_to', 'count', 'event_rate', 'woe']
    available = [c for c in display_cols if c in var_woe.columns]
    print(var_woe[available].to_string())
    
    # Monotonicity
    event_rates = var_woe['event_rate'].values
    is_mono = all(event_rates[i] <= event_rates[i+1] for i in range(len(event_rates)-1))
    print(f"\nMonotonic: {'✅ Evet' if is_mono else '❌ Hayır'}")

In [ ]:
# Excel raporunu kontrol et
import os

excel_path = os.path.join(cfg.output_folder, cfg.output_excel_path)
if os.path.exists(excel_path):
    excel_file = pd.ExcelFile(excel_path)
    
    print(f"\n📋 Excel Raporu: {excel_path}")
    print(f"Toplam {len(excel_file.sheet_names)} sheet:")
    
    # Önemli sheet'leri kontrol et
    important = ['best_model_vars_df', 'best_model_woe_df', 'models_summary']
    for sheet in important:
        if sheet in excel_file.sheet_names:
            print(f"  ✅ {sheet}")
    
    print("\n✅ Veri sözlüğü ile zenginleştirilmiş raporlar hazır!")